# GPT4All Demo
This notebook shows how to implement a chat session with [GPT4All](https://www.nomic.ai/gpt4all).

First, install the Python dependencies. You can do this from the notebook, or from the command-line before you start Jupyter.

In [1]:
%pip install --upgrade --quiet -r requirements.txt

# Restart the kernel so that it uses the new modules
get_ipython().kernel.do_shutdown(restart=True)

Note: you may need to restart the kernel to use updated packages.


{'status': 'ok', 'restart': True}

## Configuration

You can [download the GPT4All app](https://www.nomic.ai/gpt4all) and use it to download one or more models, or download model files from [Hugging Face](https://huggingface.co/) directly. If you use the app, you will need to locate the directory to which it downloads models. The location on my Mac is shown below as an example.

In [17]:
# Name of a model file compatible with GPT4All
model_name = 'Nous-Hermes-2-Mistral-7B-DPO.Q4_0.gguf'

# Location of the model file
model_directory = '/Users/ppatterson/Library/Application Support/nomic.ai/GPT4All'

# Maximum size of context window, in tokens. A higher number can produce better responses, but will consume more memory.
max_context_window = 4096

# The device on which to run the model. One of the following values:
# 'cpu': Model will run on the central processing unit.
# 'gpu': Use Metal on ARM64 macOS, otherwise the same as "kompute".
# "kompute": Use the best GPU provided by the Kompute backend.
# "cuda": Use the best GPU provided by the CUDA backend.
# "amd", "nvidia": Use the best GPU provided by the Kompute backend from this vendor.
device = 'gpu'

print(f'\nUsing {model_name} from {model_directory}, with a maximum context window of {max_context_window} tokens. Requesting the {device} device')


Using Nous-Hermes-2-Mistral-7B-DPO.Q4_0.gguf from /Users/ppatterson/Library/Application Support/nomic.ai/GPT4All, with a maximum context window of 4096 tokens. Requesting the gpu device


## Load the Large Language Model (LLM)

Note that GPT4All may not be able to use the requested device, and may not report it correctly even if it does! 

In [18]:
from gpt4all import GPT4All

print(f'Loading LLM, requesting device {device}')

model = GPT4All(
    model_name=model_name,
    model_path=model_directory,
    device=device,
    n_ctx=int(max_context_window)
)

# GPT4All may return None as the device!
print(f'Loaded LLM, running on {model.device if model.device else "cpu?"}, using the {model.backend} backend.')

Loading LLM, requesting device gpu
Loaded LLM, running on cpu?, using the metal backend.


## Configure a Prompt Template

These are the basic instructions for the LLM. Feel free to experiment by changing the prompt!

In [19]:
prompt_template = """Briefly answer the question below.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

### Question: {0}

### Response:
"""

## Process a Series of Questions within a Chat Session

GPT4All's [`chat_session`](https://docs.gpt4all.io/gpt4all_python/ref.html#gpt4all.gpt4all.GPT4All.chat_session) context manager allows the model to retain context from previous interactions when answering questions. 

In [24]:
questions = [
    'When was the Battle of Hastings?',
    'Who won it?',
    'What did they do next?'
]

with model.chat_session(prompt_template=prompt_template):
    for question in questions:
        answer = model.generate(question)
        print(f'\n{question}\n{answer}\n')


When was the Battle of Hastings?
The Battle of Hastings took place on October 14, 1066. It is considered one of the most significant battles in English history as it led to the Norman Conquest of England by William the Conqueror.


Who won it?
The Battle of Hastings was a decisive victory for William the Conqueror and his Norman forces over King Harold II and his English army. This battle marked the beginning of the Norman conquest of England, with William eventually being crowned as the first Norman king of England in 1066.


What did they do next?
After winning the Battle of Hastings, William the Conqueror continued his conquest of England by marching towards London where he was eventually recognized as king after a standoff at the city gates. He then set about consolidating his rule and implementing significant changes in governance, law, language, culture, and religion that would shape England for centuries to come.



Contrast with the same series of questions outside the context of a Chat Session. The second and third responses are somewhat random!

In [25]:
for question in questions:
    # Use format to combine the prompt template with the question  
    answer = model.generate(prompt_template.format(question))
    print(f'\n{question}\n{answer}\n')


When was the Battle of Hastings?
The Battle of Hastings took place on October 14th, 1066. It is considered one of the most significant battles in English history as it led to the Norman Conquest of England by William the Conqueror.


Who won it?
The winner of the 2019 Women's World Cup was the United States women's national soccer team. They defeated the Netherlands in a final match with a score of 2-0, winning their fourth title overall and second consecutive championship.


What did they do next?
They went outside and played in the park.

